In [1]:
import gym
import numpy as np

In [2]:
# For more info: https://github.com/openai/gym/blob/master/gym/envs/classic_control/mountain_car.py
env = gym.make('MountainCar-v0')
env.reset()

/home/tanjim/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


array([-0.59816194,  0.        ], dtype=float32)

In [3]:
print(env.reset())
print(env.observation_space.high)
print(len(env.observation_space.high))
print(env.observation_space.low)
print(env.observation_space)

[-0.41961104  0.        ]
[0.6  0.07]
2
[-1.2  -0.07]
Box([-1.2  -0.07], [0.6  0.07], (2,), float32)


```
low = np.array([self.min_position, -self.max_speed], dtype=np.float32)
high = np.array([self.max_position, self.max_speed], dtype=np.float32)
```

In [4]:
LEARNING_RATE = 0.1

DISCOUNT = 0.95
EPISODES = 25000
SHOW_EVERY = 3000

DISCRETE_OS_SIZE = [20, 20]
discrete_os_win_size = (env.observation_space.high - env.observation_space.low)/DISCRETE_OS_SIZE

q_table = np.random.uniform(low=-2, high=0, size=(DISCRETE_OS_SIZE + [env.action_space.n]))

# print(discrete_os_win_size)
# print(q_table.shape)
# print(q_table)

In [5]:
# Exploration settings
epsilon = 0.5  # not a constant, qoing to be decayed
START_EPSILON_DECAYING = 1
END_EPSILON_DECAYING = EPISODES//2
epsilon_decay_value = epsilon/(END_EPSILON_DECAYING - START_EPSILON_DECAYING)

In [6]:
def get_discrete_state(state):
    discrete_state = (state - env.observation_space.low)/discrete_os_win_size
    # print(state)
    # print(env.observation_space.low)
    # print(discrete_state)
    return tuple(discrete_state.astype(int))  # we use this tuple to look up the 3 Q values for the available actions in the q-table


In [47]:
get_discrete_state(env.reset())

[-0.59280455  0.        ]
[-1.2  -0.07]
[ 6.74661636 10.        ]


(6, 10)

In [7]:
for episode in range(EPISODES):
    discrete_state = get_discrete_state(env.reset())
    done = False

    if episode % SHOW_EVERY == 0:
        render = True
        print(episode)
    else:
        render = False

    while not done:

        #action = np.argmax(q_table[discrete_state])
        if np.random.random() > epsilon:
            # Get action from Q table
            action = np.argmax(q_table[discrete_state])
        else:
            # Get random action
            action = np.random.randint(0, env.action_space.n)


        new_state, reward, done, _ = env.step(action)

        new_discrete_state = get_discrete_state(new_state)

        if episode % SHOW_EVERY == 0:
            env.render()
        #new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)

        # If simulation did not end yet after last step - update Q table
        if not done:

            # Maximum possible Q value in next step (for new state)
            max_future_q = np.max(q_table[new_discrete_state])

            # Current Q value (for current state and performed action)
            current_q = q_table[discrete_state + (action,)]

            # And here's our equation for a new Q value for current state and action
            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)

            # Update Q table with new Q value
            q_table[discrete_state + (action,)] = new_q


        # Simulation ended (for any reson) - if goal position is achived - update Q value with reward directly
        elif new_state[0] >= env.goal_position:
            #q_table[discrete_state + (action,)] = reward
            q_table[discrete_state + (action,)] = 0

        discrete_state = new_discrete_state

    # Decaying is being done every episode if episode number is within decaying range
    if END_EPSILON_DECAYING >= episode >= START_EPSILON_DECAYING:
        epsilon -= epsilon_decay_value

env.close()

0
3000
6000
9000
12000
15000
18000
21000
24000
